In [21]:
import csv
import json
import base64
import pandas as pd
from PIL import Image
from io import BytesIO
from tqdm.autonotebook import tqdm

In [22]:
with open('./data/train/evjvqa_train_lang-short-anw.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)
    
images = train_data['images']
annotations = train_data['annotations']

In [23]:
data_dict = {
    'question_id': [],
    'image_id': [],
    'question': [],
    'answer': [],
    'objects': [],
    'image_base64': []
}

In [24]:
def transform_base64(img_path):
    img = Image.open(img_path)
    img_buffer = BytesIO()
    img.save(img_buffer, format=img.format)
    byte_data = img_buffer.getvalue()
    base64_str = base64.b64encode(byte_data) # bytes
    base64_str = base64_str.decode("utf-8") # str
    
    return base64_str

In [25]:
question_id = []
image_id = []
question = []
answer = []
objects = []
image_base64 = []

for anno in tqdm(annotations):
    if anno['language'] == 'en':
        question_id.append(anno['id'])
        image_id.append(anno['image_id'])
        question.append(anno['question'])
        answer.append(f"1.0|!+{anno['answer']}")
        objects.append(None)
        image_base64.append(transform_base64(anno['img_path']))

  0%|          | 0/23785 [00:00<?, ?it/s]

In [26]:
assert len(question_id) == len(question) == len(answer) == len(image_base64)

In [27]:
data_dict = {
    'question_id': question_id,
    'image_id': image_id,
    'question': question,
    'answer': answer,
    'objects': objects,
    'image_base64': image_base64
}

data_df = pd.DataFrame.from_dict(data_dict)

In [28]:
# with open('./data/evjvqa.tsv', 'w', encoding='utf8', newline='') as f:
#     tsv_writer = csv.writer(f, delimiter='\t', lineterminator='\n')
#     for i in range(len(question_id)):
#         tsv_writer.writerow([question_id, ima])
    

In [29]:
data_df.to_csv('./data/evjvqa.tsv', sep="\t", index=False)